<div align="center">

# RIO Airbnb - Data Cleaning Jupyter Notebook 
**Latest Update:** _17th November 2022_

</div>

#### _Import required libraries_

In [80]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statistics as sts
import datetime as dt

#### *Import initial dataset*

In [82]:
listings_df = pd.read_csv("dataset/listings.csv", index_col=False)
listings_df.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,4.581500e+04,4.581500e+04,4.581500e+04,45802.000000,45802.000000,45815.000000,45815.000000,45815.000000,0.0,42055.000000,...,34968.000000,34961.000000,34967.000000,34958.000000,34957.000000,45815.000000,45815.000000,45815.000000,45815.000000,35236.000000
mean,1.641198e+17,2.022090e+13,1.496394e+08,86.630758,116.893957,33.990937,-118.239983,4.137400,NaN,1.864749,...,4.704121,4.840750,4.823367,4.798635,4.676207,22.099596,17.293878,4.399607,0.363789,1.577828
std,2.798853e+17,0.000000e+00,1.470969e+08,494.785019,640.543978,0.190502,0.236575,2.916169,NaN,1.224218,...,0.498639,0.410545,0.438933,0.407368,0.493803,81.801488,76.833890,30.262457,3.982576,1.894040
min,1.090000e+02,2.022090e+13,5.210000e+02,1.000000,1.000000,33.338480,-118.961880,0.000000,NaN,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.392476e+07,2.022090e+13,2.413248e+07,1.000000,2.000000,33.897055,-118.396360,2.000000,NaN,1.000000,...,4.640000,4.860000,4.830000,4.760000,4.600000,1.000000,1.000000,0.000000,0.000000,0.270000
50%,4.520569e+07,2.022090e+13,9.969611e+07,3.000000,4.000000,34.044060,-118.314920,4.000000,NaN,1.000000,...,4.860000,4.970000,4.970000,4.920000,4.810000,2.000000,1.000000,0.000000,0.000000,0.890000
75%,5.492515e+17,2.022090e+13,2.503126e+08,11.000000,18.000000,34.100115,-118.086906,6.000000,NaN,2.000000,...,5.000000,5.000000,5.000000,5.000000,4.960000,9.000000,5.000000,1.000000,0.000000,2.350000
max,7.118480e+17,2.022090e+13,4.786049e+08,4031.000000,20000.000000,34.822060,-117.627790,16.000000,NaN,24.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,638.000000,638.000000,342.000000,76.000000,86.210000


#### *Remove unnecessary data columns*

In [68]:
REDUNDANT_COLUMNS = [
    'listing_url', 'scrape_id', 'last_scraped', 'source', 'host_id', 'host_url', 'host_name', 'host_location', 'host_thumbnail_url', 'host_picture_url', 'host_verifications', 
    'host_identity_verified', 'neighbourhood', 'property_type', 'bathrooms', 'calendar_updated', 'calendar_last_scraped'
]
for col in REDUNDANT_COLUMNS:
    listings_df.pop(col)   

#### *Fill in & replace selected value columns*

In [69]:
TRUE_FALSE = ['has_availability', 'instant_bookable', "host_is_superhost"]
PERCENTAGES = ['host_response_rate', "host_acceptance_rate"]
ONE_HOT_VECTORS = ['license']
NULL_VALUES = {
    'license': 'none',
    'host_response_time': 'does not respond',
    'reviews_per_month': 0,
    'bathrooms_text': "0 private baths"
}

# Fill in null values in certain columns
for nul in NULL_VALUES:
    listings_df.loc[listings_df[nul].isnull(), nul] = NULL_VALUES[nul]

# Convert all values in 'license' to categorical values
listings_df.loc[listings_df["license"] == "Exempt", "license"] = "exempted"
listings_df.loc[~listings_df['license'].str.contains('none|exempted'), 'license'] = "have"

# Convert all t/f columns to true/false & add them to the ONE_HOT_VECTORS list
for tf in TRUE_FALSE:
    listings_df.loc[listings_df[tf] == 't', tf] = f"{tf}_true"
    listings_df.loc[listings_df[tf] == 'f', tf] = f"{tf}_false"
    ONE_HOT_VECTORS.append(tf)

# Clean the 'price' column by removing the '$' and the ',' symbols as well as converting all values to "float" data type
listings_df['price'] = listings_df['price'].map(lambda x: float(x.lstrip('$').replace(",", "")))

#### *Convert categorical columns to one-hot vectors*

In [70]:
# Convert all categorical columns into one-hot vectors
for column in ONE_HOT_VECTORS:
    listings_df = listings_df.join(pd.get_dummies(listings_df[column]))
    listings_df.pop(column)

#### *Convert values from `bathrooms_text` to individual columns of `bathrooms_count` and `bathrooms_type`*

In [71]:
def bath_separator(string: str) -> dict:
    """
    Convert strings from `bathrooms_text` to counts and type  
    
    ### Parameters:
        `string`: the string we want to convert
    """
    bath_arr = string.split(" ")
    try:
        count = float(bath_arr[0])
    except ValueError:
        bath_arr = ["1"] + bath_arr
        count = float(bath_arr[0])
    
    type = " ".join(bath_arr[1:]).lower()
    
    return {
        "bathrooms_count": count,
        "bathrooms_type": type,
    }


listings_df["bathrooms_count"] = listings_df['bathrooms_text'].map(lambda x: bath_separator(x)["bathrooms_count"])
listings_df["bathrooms_type"] = listings_df['bathrooms_text'].map(lambda x: bath_separator(x)["bathrooms_type"].rstrip("s"))


#### *Drop any missing rows*

In [72]:
# Remove all listings with price of $0
listings_df = listings_df[listings_df['price'] != 0]

# Add a 'price_zscore' column
listings_df['price_zscore'] = stats.zscore(listings_df["price"])

# Remove all rows with a price z-score of more than +-2.5
listings_df = listings_df[abs(listings_df["price_zscore"]) < 2.5]

listings_df.describe()

,id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,price,minimum_nights,...,have,none,has_availability_false,has_availability_true,instant_bookable_false,instant_bookable_true,host_is_superhost_false,host_is_superhost_true,bathrooms_count,price_zscore
count,4.521400e+04,45201.000000,45201.000000,45214.000000,45214.000000,45214.000000,41484.000000,44445.000000,45214.000000,45214.000000,...,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000
mean,1.637819e+17,87.481582,117.915997,33.990167,-118.237441,4.067789,1.825523,2.280031,232.262858,17.878113,...,0.237471,0.739129,0.078560,0.921440,0.619498,0.380502,0.702327,0.296921,1.586964,-0.080546
std,2.796691e+17,497.968456,644.653461,0.190991,0.235640,2.831597,1.164331,1.716960,259.203331,30.248155,...,0.425538,0.439114,0.269053,0.269053,0.485516,0.485516,0.457240,0.456907,0.998839,0.359869
min,1.090000e+02,1.000000,1.000000,33.338480,-118.961700,1.000000,1.000000,1.000000,10.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.389128
25%,2.381052e+07,1.000000,2.000000,33.892572,-118.394588,2.000000,1.000000,1.000000,90.000000,2.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-0.278058
50%,4.517365e+07,3.000000,4.000000,34.043690,-118.312715,3.000000,1.000000,2.000000,150.000000,7.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,-0.194757
75%,5.487822e+17,11.000000,18.000000,34.099947,-118.080682,6.000000,2.000000,3.000000,264.000000,30.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,-0.036483
max,7.118480e+17,4031.000000,20000.000000,34.822060,-117.627790,16.000000,24.000000,32.000000,2085.000000,1124.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000,2.491728


<div align="center">

# *Dataset Abstraction*

#### Run the code below the view some abstract info about the data

</div>

In [65]:

prices = listings_df["price"].to_list()
col_num = len(listings_df.columns)
row_num = len(listings_df['name'])
max_revs = max(listings_df["number_of_reviews"])

max_price = max(prices)
min_price = min(prices)
mean = sts.mean(prices)
std_dev = sts.stdev(prices)

print(f"Maximum price in dataset: ${max_price:.02f}")
print(f"Minimum price in dataset: ${min_price:.02f}")
print(f"Mean of price in dataset: ${mean:.02f}")
print(f"Standard deviation of price in dataset: ${std_dev:.02f}\n")

print(f"Total # of columns after rough cleaning: {col_num}")
print(f"Total # of rows after rough cleaning: {row_num:,}\n")

print(f"Highest # of reviews for a listing: {max_revs:,}")

print("Max z-score: " + str(min(listings_df['price_zscore'])))

Maximum price in dataset: $2085.00
Minimum price in dataset: $10.00
Mean of price in dataset: $232.26
Standard deviation of price in dataset: $259.20

Total # of columns after rough cleaning: 66
Total # of rows after rough cleaning: 45,214

Highest # of reviews for a listing: 1,702
Max z-score: -0.3891275699182149


In [ ]:
city_of_la = listings_df[listings_df['neighbourhood_group_cleansed'] == "City of Los Angeles"]

<div align="center">

# *References*

</div>

#### **Export DataFrame to CSV**
Please close Excel or any other application that's opening the CSV before exporting, otherwise you'll get `[Errno 13] Permission Denied`

In [73]:
listings_df.to_csv("dataset/cleaned_listings.csv", index=False)

#### **Check for unique variables in each column**

In [74]:
COLUMN_NAME = input("Enter column name here: ")


display_list = []
for row in listings_df[COLUMN_NAME]: 
    if row not in display_list:
        display_list.append(row)


display_list

['bath',
 'shared bath',
 'private bath',
 'half-bath',
 'shared half-bath',
 'private half-bath']

#### **Find percentage in of certain values**

In [84]:
PERCENTAGES = {
    "sum": 0
}

listings_df.loc[listings_df["host_identity_verified"].isnull(), "host_identity_verified"] = "na"

for row in listings_df['host_identity_verified']:
    if row not in PERCENTAGES:
        PERCENTAGES[row] = 0
    PERCENTAGES[row] += 1
    PERCENTAGES["sum"] += 1
    
print("Percentage: " + str(
    PERCENTAGES["t"] / PERCENTAGES['sum']
))

Percentage: 0.8579067990832697
